In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import decomposition
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import euclidean_distances
import random
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from random import shuffle
import time

In [3]:
pip install git+https://github.com/samueljackson92/coranking.git


  Cloning https://github.com/samueljackson92/coranking.git to /tmp/pip-req-build-r_bt148c
  Running command git clone -q https://github.com/samueljackson92/coranking.git /tmp/pip-req-build-r_bt148c
  Created wheel for coranking: filename=coranking-0.1.1-cp36-cp36m-linux_x86_64.whl size=87029 sha256=322532b59635528b48ac2879646f8b9cbde31027a79cd15bda97a5107772631e
  Stored in directory: /tmp/pip-ephem-wheel-cache-04qwg2gc/wheels/8e/e0/11/4ae911b6fc472b8bf6087e29d861d2b798c119603adf4b93fa
Successfully built coranking


In [0]:
import coranking
from coranking.metrics import trustworthiness, continuity, LCMC


In [5]:
link1='https://drive.google.com/open?id=1K2_KnRWrKl0IfCUmKLkFerjITQt8B4lW'

fluff, id = link1.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') 
train_df  = pd.read_csv('train.csv')

1K2_KnRWrKl0IfCUmKLkFerjITQt8B4lW


In [0]:
concat_df =train_df.copy()

#columns_to_drop = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCars','Id','SalePrice']
columns_to_drop = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCars','Id','SalePrice','MasVnrArea', 'GarageYrBlt', 'LotFrontage']
concat_df.drop(columns_to_drop, axis=1, inplace=True)

nan_columns=concat_df.columns[concat_df.isna().any()].tolist()
numerical_cols = [f for f in concat_df.columns if concat_df.dtypes[f] != 'object']
containNumNans = list(set(nan_columns) & set(numerical_cols))

scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(concat_df[numerical_cols])
concat_df.loc[:,numerical_cols] = scaled_values

concat_df = pd.get_dummies(concat_df)

In [7]:
labels = train_df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(concat_df, labels, test_size=0.25, random_state=42)

GBest = ensemble.GradientBoostingRegressor(n_estimators=7000, learning_rate=0.01, max_depth=10, max_features='sqrt',min_samples_leaf=5, min_samples_split=15, loss='ls')
GB_model = GBest.fit(X_train, y_train)
preds = GBest.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))


RMSE: 24924.455372


###ISOMAP

In [0]:
#ISOMAP

from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection, neighbors)
from prettytable import PrettyTable
t_mds_10k = PrettyTable(['RMSE_onISOMAPData', 'Dimensions', 'ISOMAP_time','GBR_time' ,'total_time', 'trust_ISOMAP','cont_ISOMAP' ,'lcmc_ISOMAP'])


def MDS_Decision_Tree (training_data, components,size):

  #Reducing to mentioned components
  #print("starting mds")
  mds_start_time = time.time()
  training_data_isomap = manifold.Isomap(n_components=components).fit_transform(training_data[:size])
  mds_time = time.time() - mds_start_time
  #print("ending mds")
  
  #Applying Decision Tree
  X_train, X_test, y_train, y_test = train_test_split(training_data_isomap, labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()
  GBest = ensemble.GradientBoostingRegressor(n_estimators=7000, learning_rate=0.01, max_depth=10, max_features='sqrt',min_samples_leaf=5, min_samples_split=15, loss='ls')
  GB_model = GBest.fit(X_train, y_train)
  preds = GBest.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, preds))
  clf_time = time.time() - clf_start_time
  score=rmse
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))


  trust_mds = 0
  cont_mds = 0
  lcmc_mds = 0
  
  Q = coranking.coranking_matrix(training_data[:size], training_data_isomap)
  trust_mds = trustworthiness(Q, min_k=1, max_k=50).sum()/50
  cont_mds = continuity(Q, min_k=1, max_k=50).sum()/50
  lcmc_mds = LCMC(Q, min_k=1, max_k=50).sum()/50

  trust_mds = str(round(trust_mds, 4))
  cont_mds = str(round(cont_mds, 4))
  lcmc_mds = str(round(lcmc_mds, 4))
  
  #print(components)
  print("{}, {}, {}, {}, {}, {}, {}, {} ".format( score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds))
  print("*"*40)
  t_mds_10k.add_row([score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds])
  return

In [15]:
dimensions=266
size=1460
while dimensions >= 2:
  MDS_Decision_Tree (concat_df, dimensions,size)
  dimensions=int(dimensions/2)

44848.4306, 266, 2.8488, 40.2297, 43.0785, 0.9518, 0.9413, 0.4759 
****************************************
42604.808, 133, 2.6632, 28.3594, 31.0226, 0.9494, 0.9431, 0.4763 
****************************************
41280.8985, 66, 2.5587, 20.8311, 23.3898, 0.9448, 0.9434, 0.4649 
****************************************
39566.0649, 33, 2.5521, 14.362, 16.9141, 0.9373, 0.9419, 0.4396 
****************************************
40173.2692, 16, 2.5287, 11.6463, 14.175, 0.9232, 0.9367, 0.3943 
****************************************
42392.8359, 8, 2.2895, 7.3405, 9.63, 0.8993, 0.9279, 0.3255 
****************************************
44851.3589, 4, 2.2306, 6.5298, 8.7604, 0.8626, 0.9158, 0.2384 
****************************************
56166.1219, 2, 2.2136, 4.2041, 6.4177, 0.7932, 0.894, 0.1364 
****************************************


In [16]:
print(t_mds_10k)

+-------------------+------------+-------------+----------+------------+--------------+-------------+-------------+
| RMSE_onISOMAPData | Dimensions | ISOMAP_time | GBR_time | total_time | trust_ISOMAP | cont_ISOMAP | lcmc_ISOMAP |
+-------------------+------------+-------------+----------+------------+--------------+-------------+-------------+
|     44848.4306    |    266     |    2.8488   | 40.2297  |  43.0785   |    0.9518    |    0.9413   |    0.4759   |
|     42604.808     |    133     |    2.6632   | 28.3594  |  31.0226   |    0.9494    |    0.9431   |    0.4763   |
|     41280.8985    |     66     |    2.5587   | 20.8311  |  23.3898   |    0.9448    |    0.9434   |    0.4649   |
|     39566.0649    |     33     |    2.5521   |  14.362  |  16.9141   |    0.9373    |    0.9419   |    0.4396   |
|     40173.2692    |     16     |    2.5287   | 11.6463  |   14.175   |    0.9232    |    0.9367   |    0.3943   |
|     42392.8359    |     8      |    2.2895   |  7.3405  |    9.63    |